In [1]:
# !pip install tensorflow==2.3.0
# !pip install keras
# !pip install keras-rl2

In [2]:
from ai.tilebag.tbenv import TileBagEnv

In [3]:
tbe=TileBagEnv("http://localhost:5000", "test", "513")
tbe.runAILoop()

openai enviornment constructed: http://localhost:5000 test 513
BASE aiplayer constructed: http://localhost:5000 test 513
Connected!


<Thread(Thread-6, started 139800541771520)>

connection established
trying to join
player joined http://localhost:5000
Hmm... our turn to do something!

## Build the model that undestands our observations and actions

In [4]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [5]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [6]:
from gym.spaces import flatten_space, flatten, flatdim


In [12]:
#actions=tbe.action_space
actions=flatdim(tbe.action_space)
states=flatdim(tbe.observation_space)
model = build_model(states, actions)

In [8]:
#model.summary()

## Build the agent that will run and learn

In [9]:

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [10]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [13]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(tbe, nb_steps=50, visualize=False, verbose=1)

Training for 50 steps ...
resetting the game
Just waiting for our turn to come around
It's my turn!
yourturn data: {'game': {'gamestate': {'currplayer': {'id': '513', 'name': 'Colleen'}, 'state': 'PlaceTile', 'stateinfo': {}, 'message': 'Waiting on Colleen to play a tile'}, 'endpossible': False, 'endrequested': False, 'board': {'nrows': 9, 'ncols': 12, 'occupied': ['9D', '1E', '5E', '7E', '5F', '9H']}, 'players': [{'id': '513', 'name': 'Colleen'}, {'id': '514', 'name': 'Geoff'}, {'id': '515', 'name': 'Higgs'}, {'id': '516', 'name': 'FS&SnB'}], 'hotels': [{'name': 'Worldwide', 'tile': None, 'occupies': [], 'stocks': 25, 'price': None, 'majority': None, 'minority': None}, {'name': 'Saxxon', 'tile': None, 'occupies': [], 'stocks': 25, 'price': None, 'majority': None, 'minority': None}, {'name': 'Festival', 'tile': None, 'occupies': [], 'stocks': 25, 'price': None, 'majority': None, 'minority': None}, {'name': 'Imperial', 'tile': None, 'occupies': [], 'stocks': 25, 'price': None, 'majority

ValueError: Error when checking input: expected flatten_1_input to have 3 dimensions, but got array with shape (1, 1)